In [0]:
import dlt
from pyspark.sql.functions import col
from pyspark.sql.avro.functions import from_avro

In [0]:
# Parameters are typically passed via Pipeline settings
bootstrap = spark.conf.get('pipelines.bootstrap')
subscribe = spark.conf.get('pipelines.topic')


In [0]:
# 1) BRONZE: raw Kafka → Delta (bytes + metadata)

@dlt.table(
    name = "bronze"
    comment="Raw Kafka records (Bronze)"
    table_properties={
        quality="bronze"
    }
    )


def bronze():
    df = (spark.readStream
           .format('kafka')
           .option('kafka.bootstrap.servers', bootstrap)
           .option('subscribe', topic)
           .option('startingOffsets', 'latest')
           .option('failOnDataLoss', 'false')
           .load())
    return (df.select(
        F.col('key').alias('kafka_key'),
        F.col('value').alias('value_bytes'),
        F.col('timestamp').alias('kafka_ts'),
        F.col('topic'), 
        F.col('partition'), 
        F.col('offset')
    ))